In [114]:
import pandas as pd

In [115]:
GER_df = pd.read_csv("Resources/GER_2015.csv")
SLE_df = pd.read_csv("Resources/SLE_2015.csv")
Happiness_df = pd.read_csv("Resources/happiness_2015.csv")

In [116]:
GER_df.dropna(subset=['2015'], inplace=True)
SLE_df.dropna(subset=['2015'], inplace=True)

In [117]:
Happiness_df.rename(columns={"Country" : "Country Name"}, inplace=True)

In [118]:
GER_SLE_df = pd.merge(GER_df, SLE_df, how='inner', on='Country Name')

In [119]:
all_merged_df = pd.merge(GER_SLE_df, Happiness_df, how='inner', on='Country Name')

In [120]:
GER_cleaned = all_merged_df[["Country Name","Region","Country Code_x","Indicator Name_x","Indicator Code_x","2015_x"]].copy()
SLE_cleaned = all_merged_df[["Country Name","Region","Country Code_y","Indicator Name_y","Indicator Code_y","2015_y"]].copy()
Hap_cleaned = all_merged_df[["Country Name","Region","Happiness Rank","Happiness Score","Standard Error"]].copy()

In [121]:
GER_cleaned.rename(columns={"Country Name": "country_name",
                  "Region": "region",
                  "Country Code_x": "country_code",
                  "Indicator Name_x": "indicator_name",
                  "Indicator Code_x": "indicator_code",
                  "2015_x": "ger_value"}, inplace = True)

In [122]:
SLE_cleaned.rename(columns={"Country Name": "country_name",
                  "Region": "region",
                  "Country Code_y": "country_code",
                  "Indicator Name_y": "indicator_name",
                  "Indicator Code_y": "indicator_code",
                  "2015_y": "sle_value"}, inplace = True)

In [123]:
Hap_cleaned.rename(columns={"Country Name": "country_name",
                  "Region": "region",
                  "Happiness Rank": "happiness_rank",
                  "Happiness Score": "happiness_score",
                  "Standard Error": "standard_error"}, inplace = True)

In [124]:
GER_cleaned.to_csv("Resources/cleaned_GER_2015.csv", index=False)

In [125]:
SLE_cleaned.to_csv("Resources/cleaned_SLE_2015.csv", index=False)

In [126]:
Hap_cleaned.to_csv("Resources/cleaned_HAP_2015.csv", index=False)

In [127]:
import requests

In [128]:
url = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Afghanistan&exsentences=2&explaintext=1&format=json"
response = requests.get(url).json()
list(response['query']['pages'].values())[0]['extract']

'Afghanistan ( (listen), also pronounced /ævˈɡænᵻstæn, ævˈɡɑːnᵻstɑːn/; Pashto/Dari: افغانستان, Pashto: Afġānistān [avɣɒnisˈtɒn, ab-], Dari: Afġānestān [avɣɒnesˈtɒn]), officially the Islamic Republic of Afghanistan, is a landlocked country at the crossroads of Central and South Asia. Afghanistan is bordered by Pakistan to the east and south; Iran to the west; Turkmenistan, Uzbekistan, and Tajikistan to the north; and China to the northeast.'

In [129]:
country_list = GER_cleaned["country_name"].tolist()

In [130]:
wiki_list = []
for c in country_list:
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "exsentences": "2",
        "explaintext": "1",
        "titles": c
    }
    response = requests.get(url=url, params=params).json()
    content = list(response['query']['pages'].values())[0]['extract']
    wiki_list.append(content)

In [131]:
wiki_dict = {'country': country_list, 'wiki_info': wiki_list}
wiki_df = pd.DataFrame(wiki_dict)

In [132]:
wiki_df.rename(columns={'country': "country_name", 'wiki_info': 'wiki_info'}, inplace=True)

In [133]:
wiki_df.to_csv("Resources/cleaned_wiki.csv", index=False)

In [134]:
country_list = GER_cleaned[['country_name', 'country_code']]

In [135]:
country_list.to_csv("Resources/cleaned_list.csv", index=False)

Load dataframe into SQL

In [136]:
from sqlalchemy import create_engine

In [137]:
connection_string = "postgres:postgres@localhost:5432/ETF_Project"
engine = create_engine(f'postgresql://{connection_string}')

In [138]:
engine.table_names()

['country_list', 'ger_2015', 'sle_2015', 'hap_2015', 'wiki_info']

In [139]:
country_list.to_sql(name='country_list', con=engine, if_exists='append', index=False)

In [140]:
GER_cleaned.to_sql(name='ger_2015', con=engine, if_exists='append', index=False)

In [144]:
SLE_cleaned.to_sql(name='sle_2015', con=engine, if_exists='append', index=False)

In [145]:
Hap_cleaned.to_sql(name='hap_2015', con=engine, if_exists='append', index=False)

In [147]:
wiki_df.to_sql(name='wiki_info', con=engine, if_exists='append', index=False)